In [10]:

import numpy as np
import random

In [11]:
def add_in_news_sites(num_news_sites,fake_news_sites, total_network, number_of_people):
    '''This function adds the news sites into the network graph
    
    Parameters:
    ----------
    num_news_sites: the number of news sites in our network
    fake_news_sites: the number of fake news sites in our network
    total_network: the current matrix network (with just people)
    number_of_people: number of nodes in facebook graph
    
    Returns:
    --------
    total_network: network with fake news sites added
    news_sites_indices: indices in the total_network of the news sites
    news_sites_indices_political: array which specifies 0 (Democrat) or 1 (Republican) for each index in news_sites_indices
    fake_news_sites_indices: indices in the total_network of the fake news sites
    fake_news_sites_indices_political: array which specifies 0 (Democrat) or 1 (Republican) for each index in fake_news_sites_indices

    For example, if news_sites_indices = [233, 455, 600] and news_sites_indices_political = [0,1,1] then the news sites
    at indices 455 and 600 are republican news sites and the news site at index 233 is democratic.
    '''
    
    #3: Add in news sites and fake news sites
    n = 4039
    news_sites_indices = []
    for i in range(num_news_sites):
        news_sites_indices.append(number_of_people+i)
        total_network = np.append(total_network,[[0]]*n,axis = 1)
        total_network = np.append(total_network,[[0]*(n+1)],axis = 0)
        n += 1

    news_sites_indices_political = np.random.choice(2,num_news_sites,replace = True)

    fake_news_sites_indices = []
    for i in range(fake_news_sites):
        fake_news_sites_indices.append(news_sites_indices[-1]+1+i)
        total_network = np.append(total_network,[[0]]*n,axis = 1)
        total_network = np.append(total_network,[[0]*(n+1)],axis = 0)
        n += 1

    fake_news_sites_indices_political = np.random.choice(2,fake_news_sites,replace = True)
    
    for i in news_sites_indices:
        total_network[i,i] = 1
        
    for i in fake_news_sites_indices:
        total_network[i,i] = 1
    
    return total_network, news_sites_indices, news_sites_indices_political, fake_news_sites_indices, fake_news_sites_indices_political

In [12]:
def assign_W_weights(total_network, weight_for_significant_others,weight_given_to_news_sources,number_news_connections,number_of_people, democrats, republicans, news_sites_indices, news_sites_indices_political, fake_news_sites_indices, fake_news_sites_indices_political):
    """ This function constructs the W matrix and checks the condition that rows sum to 1 
    
    Parameters:
    -----------
    total_network: network of people/news sites
    weight_for_significant_others: % of weight given to significant others
    weight_given_to_news_sources: % of weight given to news source
    number_news_connections: number of news sites that each person is connected to
    number_of_people: number of people in network
    democrats: indicies of democratic people
    republicans: indices of republican people
    news_sites_indices: indices of news sites
    news_sites_indices_political: array which specifies 0 (Democrat) or 1 (Republican) for each index in news_sites_indices
    fake_news_sites_indices: indices of fake news sites
    fake_news_sites_indices_political: array which specifies 0 (Democrat) or 1 (Republican) for each index in fake_news_sites_indices
    
    Returns:
    --------
    W_matrix: final weight matrix
    """
    
    W_matrix = np.copy(total_network)

    for i in range(number_of_people):
        weight_given = 0
        connections = np.nonzero(W_matrix[i,:] != 0)[0]

        if i in democrats:
            my_news = [news_sites_indices[i] for i in range(len(news_sites_indices)) if news_sites_indices_political[i] == 0]
            my_fake_news = [fake_news_sites_indices[i] for i in range(len(fake_news_sites_indices)) if fake_news_sites_indices_political[i] == 0]
            all_news = np.append(my_fake_news,my_news)
            news_connections = [all_news[i] for i in np.random.choice(list(range(len(all_news))),number_news_connections,replace = False)]

        elif i in republicans:
            my_news = [news_sites_indices[i] for i in range(len(news_sites_indices)) if news_sites_indices_political[i] == 1]
            my_fake_news = [fake_news_sites_indices[i] for i in range(len(fake_news_sites_indices)) if fake_news_sites_indices_political[i] == 1]
            all_news = np.append(my_fake_news,my_news)
            news_connections = [all_news[i] for i in np.random.choice(list(range(len(all_news))),number_news_connections,replace = False)]

        for a in news_connections:
            W_matrix[i,int(a)] = weight_given_to_news_sources
            weight_given += weight_given_to_news_sources

        weight_left = 1 - weight_given

        if len(connections) > 2: 
            signif_others = [connections[i] for i in np.random.choice(list(range(len(connections))),2,replace = False)]
            for j in signif_others:
                W_matrix[i,j] = weight_for_significant_others
                weight_given += weight_for_significant_others

            for c in connections:
                if c not in signif_others:
                    W_matrix[i,c] = 0

            np.random.shuffle(connections)

            con = [c for c in connections if c not in signif_others]
            weight_left = 1 - weight_given
            to_add = weight_left/float(len(con))

            for c in con[:-1]:
                W_matrix[i,c] = to_add

            W_matrix[i,con[-1]] = 1.0 - np.sum(W_matrix, axis=1)[i]

        elif len(connections) <= 2:
            if len(connections) == 0:
                signif_others = [i]
                W_matrix[i,i] = weight_left
                weight_given = weight_left
            if len(connections) == 1:
                signif_others = [connections[0]]
                W_matrix[i,connections[0]] = weight_left
                weight_given += weight_left
            if len(connections) == 2:
                signif_others = connections
                W_matrix[i,connections[0]] = (weight_left/2.0)
                weight_given += (weight_left/2.0)
                W_matrix[i,connections[1]] = (weight_left/2.0)
                weight_given += (weight_left/2.0)


        #check that each row sums to 1 
        row_weight = np.sum(W_matrix, axis=1)[i]
        assert (round(row_weight,1) == 1.0)
         
    return W_matrix


In [13]:
def construct_W_matrix(num_news_sites,fake_news_sites,weight_for_significant_others,weight_given_to_news_sources,number_news_connections):
    """Driver function for constructing W matrix
    
    Parameters:
    -----------
    num_news_sites: number of news sites to include in network
    fake_news_sites: number of fake news sites to include in network
    weight_for_significant_others: percentage weight to give to significant others
    weight_given_to_news_sources: percentage weight to give to news sources
    number_news_connections: number of news sources each person is connected to 
    
    Returns:
    -------
    W_matrix: W matrix
    news_info: List of tuples (a,b) where a is index of news site and b is political leaning of news site (0 or 1)
    fake_news_info: List of tuples (a,b) where a is index of fake news site and b is political leaning of fake news site (0 or 1)
    democrats: indices of democratic people
    republicans: indices of republican people
    """
    
    #1: Initial set-up
    number_of_people = 4039
    total_network = np.empty([number_of_people,number_of_people])

    fb_connections = np.loadtxt('facebook_combined.txt', delimiter = " ")
    for i in fb_connections:
        total_network[int(i[0]),int(i[1])] = 1.0
        total_network[int(i[1]),int(i[0])] = 1.0
        
    #2: Split people into Republican or Democrat (Independents split evenly between the two groups)
    #40.9% call themselves liberal, 28.4% moderate, and 30.7% call themselves conservative 
    parties = list(range(number_of_people))
    np.random.shuffle(parties)
    democrats = parties[:int(len(parties)*(0.409+0.142))]
    republicans = parties[int(len(parties)*(0.409+0.142)):]
    
    total_network, news_sites_indices, news_sites_indices_political, fake_news_sites_indices, fake_news_sites_indices_political = add_in_news_sites(num_news_sites,fake_news_sites, total_network,number_of_people)
    W_matrix = assign_W_weights(total_network, weight_for_significant_others,weight_given_to_news_sources,number_news_connections, number_of_people, democrats, republicans,news_sites_indices, news_sites_indices_political, fake_news_sites_indices, fake_news_sites_indices_political)
    
    news_info = zip(news_sites_indices,news_sites_indices_political)
    fake_news_info = zip(fake_news_sites_indices,fake_news_sites_indices_political)
    
    return W_matrix, news_info, fake_news_info, democrats, republicans 

In [18]:
#######Parameters#########
num_news = 7
num_fake_news = 8
weight_for_significant_others = 0.2
weight_given_to_news_sources = 0.1
number_news_connections = 4
###########################

W_matrix, news_sites, fake_news_sites, democrats, republicans = construct_W_matrix(num_news,num_fake_news,weight_for_significant_others,weight_given_to_news_sources,number_news_connections)

"""
Tara: Heres what you need to use from my part in your part

democrats: list of indices of the democratic people, which you'll need for the X matrix
republicans: list of indices of the republican people, which you'll need for the X matrix
news_sites: tuple list of news site indices and their political affiliation, e.g. [(22,0),(13,1)] means index 22 is democratic and index 13 is republican
fake_news_sites: tuple list of fake news site indices and their political affiliation, e.g. [(22,0),(13,1)] means index 22 is democratic and index 13 is republican
"""
# Constructing matrix A, where openness is randomly asigned to people 
# news sites have 0 openness
n = 4039
s = (n+num_news+num_fake_news,n+num_news+num_fake_news)
# print (s)
A_matrix = np.zeros(s)
for (i,j), val in np.ndenumerate(A_matrix):
    if (i == j):
        A_matrix[i,j] = random.random()
# print (A_matrix)

# Constructing the new X matrix
#def construct_X_matrix


    

(4054, 4054)
[[ 0.00224215  0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.63052132  0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.35130774 ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.17129639  0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.49057914  0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.90410918]]
